In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd

driver_path = 'C:/Users/Landon Hawley/Downloads/chromedriver-win64/chromedriver.exe'
service = Service(driver_path)

driver = webdriver.Chrome(service=service)

# Open the URL
url = "https://www.espn.com/nfl/stats/player/_/stat/rushing/season/2022/seasontype/2"
driver.get(url)

time.sleep(5)

button = driver.find_element(By.LINK_TEXT, "Show More")
button.click()

# Allow time for the page to load
time.sleep(5)

button = driver.find_element(By.LINK_TEXT, "Show More")
button.click()

# Allow time for the page to load
time.sleep(5)

button = driver.find_element(By.LINK_TEXT, "Show More")
button.click()

# Allow time for the page to load
time.sleep(5)

button = driver.find_element(By.LINK_TEXT, "Show More")
button.click()

# Allow time for the page to load
time.sleep(5)

button = driver.find_element(By.LINK_TEXT, "Show More")
button.click()

# Allow time for the page to load
time.sleep(5)

button = driver.find_element(By.LINK_TEXT, "Show More")
button.click()

# Allow time for the page to load
time.sleep(5)

button = driver.find_element(By.LINK_TEXT, "Show More")
button.click()

# Allow time for the page to load
time.sleep(5)

# Get the page source after JavaScript has loaded
html_content = driver.page_source

driver.quit()

In [3]:
soup = BeautifulSoup(html_content, "lxml")

namerank_columns = ['Rank', 'Name/Team']
namerank_df = pd.DataFrame(columns = namerank_columns)

namerank_div = soup.find(name = 'div', attrs = {'class' : 'ResponsiveTable ResponsiveTable--fixed-left mt4 Table2__title--remove-capitalization'})
table = namerank_div.find("table") 
tbody = table.find("tbody")  # Get the tbody element
rows = tbody.find_all("tr")  # Find all table rows

for row in rows:
    cols = row.find_all("td")  # Get all table data cells
    data = [col.text.strip() for col in cols]  # Extract text from each column
    namerank_series = pd.Series(data, index = namerank_columns)
    namerank_df = pd.concat([namerank_df, namerank_series.to_frame().T], ignore_index=True)

In [4]:
import re


team_abbrs = [
    "ARI", "ATL", "BAL", "BUF", "CAR", "CHI", "CIN", "CLE", "DAL", "DEN",
    "DET", "GB", "HOU", "IND", "JAX", "KC", "LAC", "LAR", "LV", "MIA",
    "MIN", "NE", "NO", "NYG", "NYJ", "PHI", "PIT", "SEA", "SF", "TB",
    "TEN", "WSH", "OAK"
]


team_pattern = r'(' + '|'.join(team_abbrs) + r')$'


def split_string_first(s):
    match = re.search(team_pattern, s)
    if match:
        return s[:match.start()]
    return s


def split_string_second(s):
    match = re.search(team_pattern, s)
    if match:
        return match.group()
    return ''


namerank_df['Name'] = namerank_df['Name/Team'].apply(split_string_first)
namerank_df['Team'] = namerank_df['Name/Team'].apply(split_string_second)
namerank_df.drop('Name/Team', inplace=True, axis=1)

In [5]:
stats_columns = ['Position', 'Games Played', 'Rushing Attempts', 'Rushing Yards',
                 'Avg Yards per Rush Attempt', 'Long Rushing', '20+ Yard Rushes', 'Rushign Touchdowns',
                 'Rushing Yards per Game', 'Rushing Fumbles', 'Rushing Fumbles Lost', 'Rushing First Downs']
stats_df = pd.DataFrame(columns = stats_columns)

stats_div = soup.find(name = 'div', attrs = {'class' : 'Table__Scroller'})
table = stats_div.find("table") 
tbody = table.find("tbody")  # Get the tbody element
rows = tbody.find_all("tr")  # Find all table rows

for row in rows:
    cols = row.find_all("td")  # Get all table data cells
    data = [col.text.strip() for col in cols]  # Extract text from each column
    stats_series = pd.Series(data, index = stats_columns)
    stats_df = pd.concat([stats_df, stats_series.to_frame().T], ignore_index=True)

stats_df

,Position,Games Played,Rushing Attempts,Rushing Yards,Avg Yards per Rush Attempt,Long Rushing,20+ Yard Rushes,Rushign Touchdowns,Rushing Yards per Game,Rushing Fumbles,Rushing Fumbles Lost,Rushing First Downs
0,RB,17,340,"1,653",4.9,86,7,12,97.2,2,0,93
1,RB,16,349,"1,538",4.4,56,10,13,96.1,4,2,65
2,RB,17,302,"1,525",5.1,41,13,12,89.7,1,1,69
3,RB,16,295,"1,312",4.4,68,9,10,82.0,1,0,62
4,RB,17,259,"1,269",4.9,40,9,11,74.6,1,1,62
...,...,...,...,...,...,...,...,...,...,...,...,...
361,WR,12,2,-7,-3.5,3,0,0,-0.6,0,0,0
362,TE,14,2,-8,-4.0,-2,0,0,-0.6,0,0,0
363,WR,14,2,-11,-5.5,7,0,0,-0.8,1,1,2
364,WR,17,2,-15,-7.5,-5,0,0,-0.9,0,0,0


In [6]:
total_stats = pd.concat([namerank_df, stats_df], axis=1)
total_stats['Year'] = 2022
total_stats

,Rank,Name,Team,Position,Games Played,Rushing Attempts,Rushing Yards,Avg Yards per Rush Attempt,Long Rushing,20+ Yard Rushes,Rushign Touchdowns,Rushing Yards per Game,Rushing Fumbles,Rushing Fumbles Lost,Rushing First Downs,Year
0,1,Josh Jacobs,LV,RB,17,340,"1,653",4.9,86,7,12,97.2,2,0,93,2022
1,2,Derrick Henry,TEN,RB,16,349,"1,538",4.4,56,10,13,96.1,4,2,65,2022
2,3,Nick Chubb,CLE,RB,17,302,"1,525",5.1,41,13,12,89.7,1,1,69,2022
3,4,Saquon Barkley,NYG,RB,16,295,"1,312",4.4,68,9,10,82.0,1,0,62,2022
4,5,Miles Sanders,PHI,RB,17,259,"1,269",4.9,40,9,11,74.6,1,1,62,2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,361,Jahan Dotson,WSH,WR,12,2,-7,-3.5,3,0,0,-0.6,0,0,0,2022
362,363,David Njoku,CLE,TE,14,2,-8,-4.0,-2,0,0,-0.6,0,0,0,2022
363,364,Jakobi Meyers,NE,WR,14,2,-11,-5.5,7,0,0,-0.8,1,1,2,2022
364,365,DeAndre Carter,LAC,WR,17,2,-15,-7.5,-5,0,0,-0.9,0,0,0,2022


In [7]:
total_stats.drop(total_stats[total_stats['Name'].str.contains('/')].index, inplace = True)
total_stats

,Rank,Name,Team,Position,Games Played,Rushing Attempts,Rushing Yards,Avg Yards per Rush Attempt,Long Rushing,20+ Yard Rushes,Rushign Touchdowns,Rushing Yards per Game,Rushing Fumbles,Rushing Fumbles Lost,Rushing First Downs,Year
0,1,Josh Jacobs,LV,RB,17,340,"1,653",4.9,86,7,12,97.2,2,0,93,2022
1,2,Derrick Henry,TEN,RB,16,349,"1,538",4.4,56,10,13,96.1,4,2,65,2022
2,3,Nick Chubb,CLE,RB,17,302,"1,525",5.1,41,13,12,89.7,1,1,69,2022
3,4,Saquon Barkley,NYG,RB,16,295,"1,312",4.4,68,9,10,82.0,1,0,62,2022
4,5,Miles Sanders,PHI,RB,17,259,"1,269",4.9,40,9,11,74.6,1,1,62,2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,361,Jahan Dotson,WSH,WR,12,2,-7,-3.5,3,0,0,-0.6,0,0,0,2022
362,363,David Njoku,CLE,TE,14,2,-8,-4.0,-2,0,0,-0.6,0,0,0,2022
363,364,Jakobi Meyers,NE,WR,14,2,-11,-5.5,7,0,0,-0.8,1,1,2,2022
364,365,DeAndre Carter,LAC,WR,17,2,-15,-7.5,-5,0,0,-0.9,0,0,0,2022


In [8]:
total_stats.to_csv("Rushing_2022.csv", index = False)
print("Data saved to Rushing_2022.csv")

Data saved to Rushing_2022.csv
